In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import csv
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from PIL.ImageColor import colormap
from cartopy import crs as ccrs
from cartopy import feature as cfeature
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.interpolate import griddata
from scipy.spatial import cKDTree

#from joblib import Parallel, delayed
from typing import Tuple

In [ ]:
# Load the data
equipment = pd.read_csv('../data/equipment.csv', delimiter=',')

In [ ]:
equipment.info()

In [ ]:
#equipment.drop(columns=['Ist'], inplace=True)

In [ ]:
equipment['Postleitzahl'] = equipment['Postleitzahl'].astype(str)

def expand_plz(postleitzahl: str) -> str:
    if len(postleitzahl) < 5:
        return '0' + postleitzahl
    return postleitzahl

equipment['Postleitzahl'] = equipment['Postleitzahl'].apply(expand_plz)

In [ ]:
equipment.head()

In [ ]:
# CSV-Datei mit Postleitzahlen und Koordinaten einlesen
plz_koordinaten = {}
with open('../data/plz_geocoord.csv', mode='r') as infile:
    reader = csv.reader(infile)
    next(reader)  # Überspringe die Kopfzeile
    for rows in reader:
        plz, lat, lon = rows
        plz_koordinaten[plz] = (float(lat), float(lon))


In [ ]:
# Funktion zum Nachschlagen der Koordinaten
def get_coordinates(postleitzahl: str) -> Tuple[float, float]:
    return plz_koordinaten.get(postleitzahl, (np.nan, np.nan))

# Neue Spalten für Latitude und Longitude hinzufügen
equipment[['lat', 'lon']] = equipment['Postleitzahl'].apply(get_coordinates).apply(pd.Series)

In [ ]:
equipment.head()

In [ ]:
equipment.dropna(inplace=True)

In [ ]:
equipment.info()

In [ ]:
equipment.rename({'Anzahl Sätze': 'incident_count'}, axis='columns', inplace=True)

In [ ]:
with open('../data/equipment.csv', mode='w') as outfile:
    equipment.to_csv(outfile, index=False)

In [ ]:
# Create Contour graph
lat_values = np.linspace(equipment['lat'].min(), equipment['lat'].max(), 100)
lon_values = np.linspace(equipment['lon'].min(), equipment['lon'].max(), 100)
lat_grid, lon_grid = np.meshgrid(lat_values, lon_values)

# Interpolate Incident values for the grid (for visualization purposes)
incident_grid = griddata((equipment['lat'], equipment['lon']), equipment['incident_count'], (lat_grid, lon_grid), method='linear')

plt.contourf(lon_grid, lat_grid, incident_grid, cmap='viridis')
plt.colorbar()

In [ ]:
plt.figure(figsize=(5,11.8/2))
plt.scatter(equipment["lon"], equipment["lat"], s=5)
plt.savefig("../plots/equipment_scatter.svg", format='svg')
plt.show()

In [ ]:
X_train = equipment[["Equipment", "lat", "lon", "incident_count"]].copy()
X_train.dropna(axis=0, inplace=True)

X_train.info()

In [ ]:
print(np.round(X_train.max(), 2), "\n", np.round(X_train.min(), 2))

In [ ]:
# Separate the 'equipment_id' column
equipment_id = X_train['Equipment']
features = X_train.drop(columns=['Equipment'])


In [ ]:
# Scale only the feature columns
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
features_scaled = pd.DataFrame(features_scaled, columns=features.columns)

# Combine the scaled features with the 'equipment_id' column
X_train_scaled = pd.concat([equipment_id.reset_index(drop=True), features_scaled], axis=1)

In [ ]:
# Define the parameter grid
eps_values = np.arange(0.5, 2, 0.1)
min_samples_values = range(75, 200, 10)

print("Number of variations to test", len(eps_values) * len(min_samples_values))

In [ ]:
def evaluate_dbscan(eps: int, min_samples: int, x: pd.DataFrame) -> Tuple[int, int, float, int]:
    dbscan_gs = DBSCAN(eps=eps, min_samples=min_samples)
    labels_gs = dbscan_gs.fit_predict(x)
    
    # Silhouette Score requires at least 2 clusters, however, 2 clusters is not useful for our case
    if len(set(labels_gs)) > 2:
        score = silhouette_score(x, labels_gs)
    else:
        score = -1  # Invalid score if less than 2 clusters are found
    
    return eps, min_samples, score, len(set(labels_gs))

In [ ]:
# # Perform parallel grid search
# results = Parallel(n_jobs=-1)(delayed(evaluate_dbscan)(eps, min_samples, X_train[["lat", "lon", "weather_score"]])
#                               for eps in eps_values
#                               for min_samples in min_samples_values)

# # Convert results to a DataFrame for easier analysis
# results_df = pd.DataFrame(results, columns=['eps', 'min_samples', 'score', 'n_clusters'])

# # Display results
# print(results_df)

In [ ]:
# # Run DBSCAN
# # Identify the best combination of parameters
# best_result = results_df.loc[results_df['score'].idxmax()]
# print(best_result)

# dbscan = DBSCAN(eps=best_result["eps"], min_samples=int(best_result["min_samples"]))
# # dbscan = DBSCAN(eps=0.3, min_samples=75)
# labels = dbscan.fit_predict(X_train[['lat', 'lon', 'weather_score']])


In [ ]:
# from sklearn.cluster import KMeans

# # Define the parameter grid
# n_clusters_values = range(2, 10)

# def evaluate_kmeans(n_clusters: int, X: pd.DataFrame) -> Tuple[int, float]:
#     kmeans = KMeans(n_clusters=n_clusters)
#     kmeans.fit(X)
#     labels = kmeans.predict(X)
#     score = silhouette_score(X, labels)
#     return (n_clusters, score)

# # Perform parallel grid search
# results = Parallel(n_jobs=-1)(delayed(evaluate_kmeans)(n_clusters, X_train[["lat", "lon", "weather_score"]])
#                               for n_clusters in n_clusters_values)

# # Convert results to a DataFrame for easier analysis
# results_df = pd.DataFrame(results, columns=['n_clusters', 'score'])

# # Display results
# print(results_df)

In [ ]:
dbscan = DBSCAN(eps=0.25, min_samples=75)
labels = dbscan.fit_predict(X_train_scaled[['lat', 'lon', 'incident_count']])

In [ ]:
num_clusters = len(set(labels) - {-1})

print(f"Number of clusters: {num_clusters}")

In [ ]:
X_train_scaled.loc[:, 'cluster'] = labels

# Ensure 'Equipment' is of the same type in both DataFrames
X_train['Equipment'] = X_train['Equipment'].astype(int)
X_train_scaled['Equipment'] = X_train_scaled['Equipment'].astype(int)

X_train = X_train.merge(X_train_scaled[['Equipment', 'cluster']], on='Equipment')

In [ ]:
noise_data = X_train[X_train['cluster'] == -1]
non_noise_data = X_train[X_train['cluster'] != -1]

# Create a 3D scatter plot using Plotly Graph Objects
fig = go.Figure()

# Add trace for non-noise clusters
fig.add_trace(go.Scatter3d(
    x=non_noise_data['lon'],
    y=non_noise_data['lat'],
    z=non_noise_data['incident_count'],
    mode='markers',
    marker=dict(
        size=8,
        color=non_noise_data['cluster'],  # Color by cluster
        colorscale='Viridis',              # Color scale
        opacity=0.8,
        colorbar=dict(title='Cluster')
    ),
    text=non_noise_data['incident_count'],  # Hover text
    hovertemplate='<b>Incident count:</b> %{text}<br><b>Lat:</b> %{y}<br><b>Lon:</b> %{x}'
))

# Add trace for noise cluster
fig.add_trace(go.Scatter3d(
    x=noise_data['lon'],
    y=noise_data['lat'],
    z=noise_data['incident_count'],
    mode='markers',
    marker=dict(
        size=8,
        color='purple',  # Color for noise points
        opacity=0.01,
    ),
    text=noise_data['incident_count'],  # Hover text
    hovertemplate='<b>Incident count:</b> %{text}<br><b>Lat:</b> %{y}<br><b>Lon:</b> %{x}'
))

# Update layout for better visualization
fig.update_layout(
    title='3D Scatter Plot of Equipment Clusters in Germany',
    scene=dict(
        xaxis=dict(title='Longitude'),
        yaxis=dict(title='Latitude'),
        zaxis=dict(title='Incident count'),
        aspectmode='cube'  # Ensure aspect ratio is equal
    )
)

fig.show()

# Save the plot as an HTML file
fig.write_html('../plots/clusters_incidents.html')

In [ ]:
# Calculate centroid of each cluster
cluster_centers = X_train.groupby('cluster')[['lat', 'lon', 'incident_count']].mean().reset_index()

reduce_maintenance = []
increase_maintenance = []

for centroid in cluster_centers.itertuples():
    print(centroid)
    if centroid.cluster == -1:
        continue
    if centroid.incident_count >= 5:
        increase_maintenance.append(centroid)
    elif centroid.incident_count < 5:
        reduce_maintenance.append(centroid)

reduce_maintenance = pd.DataFrame(reduce_maintenance)
increase_maintenance = pd.DataFrame(increase_maintenance)

In [ ]:
reduce_maintenance.info()

In [ ]:
# Create a 3D scatter plot using Plotly Graph Objects
fig = go.Figure()

reduce_maintenance_equipment = X_train[X_train['cluster'].isin(reduce_maintenance['cluster'])]
increase_maintenance_equipment = X_train[X_train['cluster'].isin(increase_maintenance['cluster'])]

fig.add_trace(go.Scatter3d(
    x=reduce_maintenance_equipment['lon'],
    y=reduce_maintenance_equipment['lat'],
    z=reduce_maintenance_equipment['incident_count'],
    mode='markers',
    marker=dict(
        size=8,
        color='green',
        colorscale='Viridis',              # Color scale
        opacity=0.8,
        colorbar=dict(title='Cluster')
    ),
    text=reduce_maintenance_equipment['incident_count'],  # Hover text
    hovertemplate='<b>Incident count:</b> %{text}<br><b>Lat:</b> %{y}<br><b>Lon:</b> %{x}'
))

fig.add_trace(go.Scatter3d(
    x=increase_maintenance_equipment['lon'],
    y=increase_maintenance_equipment['lat'],
    z=increase_maintenance_equipment['incident_count'],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        colorscale='Viridis',              # Color scale
        opacity=0.8,
        colorbar=dict(title='Cluster')
    ),
    text=increase_maintenance_equipment['incident_count'],  # Hover text
    hovertemplate='<b>Incident count:</b> %{text}<br><b>Lat:</b> %{y}<br><b>Lon:</b> %{x}'
))


# Update layout for better visualization
fig.update_layout(
    title='3D Scatter Plot of Equipment Clusters in Germany',
    scene=dict(
        xaxis=dict(title='Longitude'),
        yaxis=dict(title='Latitude'),
        zaxis=dict(title='Incident count'),
        aspectmode='cube'  # Ensure aspect ratio is equal
    )
)

fig.show()

# Save the plot as an HTML file
fig.write_html('../plots/result_incidents.html')

In [ ]:
# Assign colors
increase_maintenance_equipment.loc[:, 'color'] = 'red'
reduce_maintenance_equipment.loc[:, 'color'] = 'green'

combined = pd.concat([reduce_maintenance_equipment, increase_maintenance_equipment])

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})

# Add geographical features
ax.add_feature(cfeature.BORDERS, linestyle='-')
ax.add_feature(cfeature.LAND, facecolor='white')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
ax.add_feature(cfeature.COASTLINE, zorder=5)

# Plot data points
for color, group in combined.groupby('color'):
    print(f"Plotting color: {color} with {len(group)} points")  # Debugging statement
    ax.scatter(group['lon'], group['lat'], color=color, s=100, alpha=0.7, transform=ccrs.PlateCarree())

# Set plot title and extent
ax.set_title('Equipment Clusters in Germany')
ax.set_extent([5, 15, 47, 55], crs=ccrs.PlateCarree())

# Add legend
ax.legend(['Reduce Maintenance', 'Increase Maintenance'], loc='upper left')

# Save the plot as an image
plt.savefig('../plots/cluster_plot_incidents.svg', bbox_inches='tight')

# Show the plot
plt.show()